In [ ]:
from groq import Groq
import os
import re

client = Groq(
    api_key="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"
)
for i in os.listdir("Corrected"):
    with open(f'Corrected/{i}', 'r' , encoding="utf8") as resume_file:
        c=10000
        resume_f = resume_file.read()
        for j in range(c , len(resume_f) - 20000 , 5500  ):
            resume = resume_f[j : j+5000]
            completion = client.chat.completions.create(
                model="llama3-70b-8192",
                messages=[
                    {
                        "role": "user",
                        "content": """ Please extract the numeric values for the following parameters from the provided text:
                                    pH,Calcaire Total,Matière Organique (MO),Capacité d'Échange Cationique (CEC),Conductivité Électrique (CE),Densité Apparente (Da)
                                    Additionally, include the study location, which corresponds to a wilaya in Algeria, based on the provided latitude and longitude in the text, as well as the depth of the soil.
                                    Return the results in JSON format, structured as follows:
                                    {
                                        "location": {
                                            "wilaya": "Name of Wilaya",
                                            "latitude": "latitude_value",
                                            "longitude": "longitude_value"
                                        },
                                        "parameters": {
                                            "pH": numeric_value,
                                            "Calcaire Total": numeric_value,
                                            "Matière Organique (MO)": numeric_value,
                                            "Capacité d'Échange Cationique (CEC)": numeric_value,
                                            "Conductivité Électrique (CE)": numeric_value,
                                            "Densité Apparente (Da)": numeric_value
                                        },
                                        "depth": "depth_value"
                                    }
                                here is the resume
                                """+resume
                    },
                    {
                        "role": "assistant",
                        "content": "return a json response containe the needed data from the resume"
                    }
                ],
                temperature=1,
                max_tokens=1024,
                top_p=1,
                stream=False,
                stop=None,
            )

            output_file_name = f"f.md"    
            with open(output_file_name, 'a') as output_file:
                try : 
                    matches = re.findall(r'```(.*?)```', completion.choices[0].message.content, re.DOTALL)
                    output_file.write(f"this output is for the doc : {i} here is his output \n  {matches[0]}")
                except Exception as e : 
                    pass
        print(f"doc {i} is done")

transforming model responses to dataframe

1. cleaned json version

In [3]:
def remove_comments(json_text):
    cleaned_text = re.sub(r'//.*', '', json_text)
    return cleaned_text
file_path = 'final.json'

with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()
cleaned_content = remove_comments(content)
with open(file_path, 'w', encoding='utf-8') as file:
    content = file.write(cleaned_content)


2. transforme to csv

In [ ]:
import pandas as pd
import json

with open('final.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

rows = []
for index , i in enumerate(data):
    try : 
        row = {
            "Wilaya": i["location"]["wilaya"],
            "Lat": i["location"]["latitude"],
            "Lon": i["location"]["longitude"],
            "Ph": i["parameters"]["pH"],
            "Calcaire Total": i["parameters"]["Calcaire Total"],
            "Matiere Organique (MO)": i["parameters"]["Matiere Organique (MO)"],
            "Capacite d'echange Cationique (CEC)": i["parameters"]["Capacite d'echange Cationique (CEC)"],
            "Conductivite electrique (CE)": i["parameters"]["Conductivite electrique (CE)"],
            "Densite Apparente (Da)": i["parameters"]["Densite Apparente (Da)"],
            "Depth": i["depth"]
        }
        rows.append(row)
    except Exception as e : 
        print(f"in position : {index} ,{e}")
        print(i)

df = pd.DataFrame(rows)

In [47]:
df

,Wilaya,Lat,Lon,Ph,Calcaire Total,Matiere Organique (MO),Capacite d'echange Cationique (CEC),Conductivite electrique (CE),Densite Apparente (Da),Depth
0,Jijel,None,None,None,None,None,None,None,None,None
1,Jijel,36.7833,5.7667,7.8,14.15,2.53,24.58,0.25,1.45,0-20 cm
2,,,,None,None,None,None,None,None,
3,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND,NOT_FOUND
4,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
1142,El-Hodna,34.8833e N,5.7167e E,7.83,18.25,1.15,15.32,1.21,1.48,0-90 cm and 0-150 cm
1143,"(latitude: , longitude: )",,,None,None,None,None,None,None,"0-90cm, 0-150cm"
1144,,,,,,,,,,
1145,Tiaret,35.3843e N,1.7947e E,8.52,2.48,21.99,2.47,2.21,1.71,not provided


3. cleaning the dataset column per column